In [ ]:
# Glenda and Chris' Awesome Code!

In [ ]:
#added by glenda

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import country_converter as coco
from sqlalchemy import create_engine
import gmaps

# Import API key
from api_keys import weather_api_key
from api_keys import google_key
from config import username
from config import password


gmaps.configure(api_key = google_key)


# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

Store CSV files into dataframes

In [ ]:
# EXTRACTING data from NOAA's HURDAT2 available at this website:
# https://www.kaggle.com/noaa/hurricane-database

csv_file = "Resources/atlantic.csv"
atlantic_data_df = pd.read_csv(csv_file)
atlantic_data_df.head()

In [ ]:
# TRANSFORMING the Atlantic Ocean data by dropping columns that were largely unused

new_atlantic_data_df = atlantic_data_df[["ID","Name","Date","Time","Status","Latitude","Longitude","Maximum Wind","Minimum Pressure"]]
new_atlantic_data_df

In [ ]:
# EXTRACTING data from NOAA's HURDAT2 available at this website:
# https://www.kaggle.com/noaa/hurricane-database

csv_file = "Resources/pacific.csv"
pacific_data_df = pd.read_csv(csv_file)
pacific_data_df.head()

In [ ]:
# TRANSFORMING the Pacific Ocean data by dropping columns that were largely unused

new_pacific_data_df = pacific_data_df[["ID","Name","Date","Time","Status","Latitude","Longitude","Maximum Wind","Minimum Pressure"]]
new_pacific_data_df

In [ ]:
# TRANSFORMING a combination of Atlantic Ocean and Pacific Ocean data into a single dataframe

atlantic_and_pacific_data = pd.concat([new_atlantic_data_df, new_pacific_data_df])
atlantic_and_pacific_data

Data Transformation

In [ ]:
# CONFIRMING the datatypes that were Extracted

atlantic_and_pacific_data.dtypes

In [ ]:
# TRANSFORMING by filtering the data from the CSV files to shrink the data to process

# Exclude any data from hurricane season prior to the year 2000  (Date >= year 2000)
atlantic_and_pacific_2000 = atlantic_and_pacific_data.loc[atlantic_and_pacific_data["Date"] >= 20000000, :]

clean_atlantic_and_pacific_df = atlantic_and_pacific_2000

# Exclude any data from hurricane season that was less than 60 mph (Maximum Wind >= 60)
clean_atlantic_and_pacific_df = atlantic_and_pacific_2000.loc[atlantic_and_pacific_2000["Maximum Wind"] >= 60, :]
clean_atlantic_and_pacific_df.shape

In [ ]:
# TRANSFORM the data by converting from a user-friendly format of latitude and longitude (eg. 51N, 101W)
# to one that was more commonly used with APIs (e.g. 51, -101)

# convert latitude and longitude values to decimal format
def cardinal2negative(s):
    degrees = float(s[:-1])
    cardinal = s[len(s)-1:]
    
    if cardinal in ('s','S','w','W'):
        degrees *= -1

    return degrees

# Perform this step to skip data copying warning from python
clean_atlantic_and_pacific_df = clean_atlantic_and_pacific_df.copy()
lat_series = clean_atlantic_and_pacific_df["Latitude"].copy().apply(cardinal2negative)
lng_series = clean_atlantic_and_pacific_df['Longitude'].copy().apply(cardinal2negative)

clean_atlantic_and_pacific_df["New Latitude"] = lat_series
clean_atlantic_and_pacific_df['New Longitude'] = lng_series

# TRANSFORM the data by removing the leading whitespace in-front of the
# hurricane's Name and Status left-over from CSV
clean_atlantic_and_pacific_df["Name"] = clean_atlantic_and_pacific_df["Name"].str.strip()
clean_atlantic_and_pacific_df["Status"] = clean_atlantic_and_pacific_df["Status"].str.strip()
clean_atlantic_and_pacific_df

clean_atlantic_and_pacific_df


In [ ]:
##############################################################################
# THIS STEP WILL TAKE A FEW MINUTES TO RUN. PLEASE BE PATIENT (coco is slow) #
##############################################################################

# TRANSFORM the data by using the citipy API to determine the nearest city 
# for each lat lng pair and add it to the dataframe

unique_cities_list = []
city_list = []
country_list = []

# Create a set of latitude and longitude combinations to process over in a simple for-loop
lats = clean_atlantic_and_pacific_df["New Latitude"]
lngs = clean_atlantic_and_pacific_df["New Longitude"]
lat_lngs = zip(lats, lngs)

# EXTRACT from the citipy API to identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    
    # TRANSFORM the data by uppercasing cities and converting country codes to country names
    city = city.title()
    country = coco.convert(names=country, to='name_short')

    city_list.append(city)
    country_list.append(country)

    if (country == "not found"):
        continue ### We want to skip this location if we couldn't convert the country code

    city_country_pair = f"{city}, {country}"
    
    # If the city is unique, then add it to our unique cities list
    if city_country_pair not in unique_cities_list:
        unique_cities_list.append(city_country_pair)


# Add the city and country pair to the dataframe here:
clean_atlantic_and_pacific_df["City"] = city_list
clean_atlantic_and_pacific_df["Country"] = country_list

### Warning for the Netherland's Antilles island's country code "AN" for not being in citipy

In [ ]:
#clean_atlantic_and_pacific_df.head()

In [ ]:
# TRANSFORM the data by filtering out any location where the country code could not be converted
clean_atlantic_and_pacific_df = clean_atlantic_and_pacific_df[clean_atlantic_and_pacific_df["Country"] != "not found"]

# Print the city count to confirm sufficient count
len(unique_cities_list)
unique_cities_list

In [ ]:
#clean_atlantic_and_pacific_df.head()

In [ ]:
clean_atlantic_and_pacific_df.shape

In [ ]:
# LOAD - Perform the loading of the data into the sql server here:
rds_connection_string = f'{username}:{password}@localhost:5432/hurricanes'
engine = create_engine(f'postgresql://{rds_connection_string}')

# Confirm the table name in the database:
engine.table_names()


In [ ]:
column_names = clean_atlantic_and_pacific_df.columns
column_names

In [ ]:
# TRANSFORM the data by converting the column names to something more palatable by postgres

# Consrtruct an empty list to be populated soon
sql_column_names = []

# Make a deep copy of the hurricane dataframe
to_sql_df = clean_atlantic_and_pacific_df.copy()

# Loop over all the column names
for name in column_names:
    
    # And append them to a list of lowercased column names
    sql_column_names.append(name.lower())
    
# Set all the column names in the deep-copy dataframe to their lowercase equivalents
to_sql_df.columns = sql_column_names

# TRANSFORM
# Rename the columns that have spaces in the middle to use underscores instead
to_sql_df.rename(columns={"date":"date_stamp", 
                          "time":"time_stamp", 
                          "maximum wind":"max_wind",
                          "minimum pressure":"min_pressure",
                          "new latitude":"new_latitude",
                          "new longitude":"new_longitude"} , inplace=True)

to_sql_df.head()

In [ ]:
to_sql_df.shape

In [ ]:
# LOAD the data INTO the SQL server database
to_sql_df.to_sql(name='hurricanes', con=engine, if_exists='replace', index=True)

In [ ]:
# LOAD/EXTRACT the data FROM the SQL database
katrina_from_sql_df = pd.read_sql_query("select * from hurricanes where name = 'KATRINA'", con=engine)
coords = katrina_from_sql_df[["new_latitude", "new_longitude"]]

# EXTRACT Google Maps data
fig = gmaps.figure(zoom_level=5, center=(27.5,-85))

#heat_layer = gmaps.heatmap_layer(coords, dissipating=False, max_intensity=100, point_radius=3)
#fig.add_layer(heat_layer)

# TRANSFORM the EXTRACTED data into a Google Maps symbol layer
hurricane_katrina_layer = gmaps.symbol_layer(
    coords, fill_color='red', stroke_color='red', scale=2
)
fig.add_layer(hurricane_katrina_layer)
fig


In [ ]:
# LOAD/EXTRACT the data FROM the SQL database
all_hurricanes_from_sql_df = pd.read_sql_query('select * from hurricanes where max_wind >= 115', con=engine)
coords = all_hurricanes_from_sql_df[["new_latitude", "new_longitude"]]

# EXTRACT Google Maps data
fig = gmaps.figure(zoom_level=3, center=(25,-75))

# TRANSFORM the EXTRACTED data into a Google Maps symbol layer
all_hurricanes_layer = gmaps.symbol_layer(
    coords, fill_color='green', stroke_color='green', scale=2
)
fig.add_layer(all_hurricanes_layer)
fig.add_layer(hurricane_katrina_layer)
fig

In [ ]:
# LOAD/EXTRACT the data FROM the SQL database
all_hurricanes_from_sql_df = pd.read_sql_query('select * from hurricanes where max_wind >= 115', con=engine)
coords = all_hurricanes_from_sql_df[["new_latitude", "new_longitude"]]
winds = all_hurricanes_from_sql_df["max_wind"]

winds.describe()

In [ ]:
# EXTRACT Google Maps data
fig = gmaps.figure(zoom_level=3, center=(25,-75))

# TRANSFORM the EXTRACTED data into a Google Maps heatmap layer
heat_layer = gmaps.heatmap_layer(coords, weights=winds, dissipating=True, max_intensity=150, point_radius=7)

fig.add_layer(heat_layer)
fig.add_layer(hurricane_katrina_layer)
fig